In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Cargar y analizar dataset original
df = pd.read_csv('heart_data_aumentado.csv')
# df = pd.get_dummies(df, columns=['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope'], drop_first=True)

# Separar por clase para simular pacientes sanos y con enfermedad por separado
df_sano = df[df['HeartDisease'] == 0]
df_enfermo = df[df['HeartDisease'] == 1]

def generar_sinteticos_mejorado(df_base, n=500):
    """
    Genera datos sintéticos realistas respetando los tipos de variable.
    """
    df_out = pd.DataFrame()

    for col in df_base.columns:
        valores_unicos = df_base[col].nunique()

        if valores_unicos == 2 and df_base[col].dropna().isin([0, 1]).all():
            # Binaria
            p = df_base[col].mean()
            df_out[col] = np.random.binomial(1, p, n).astype(int)

        elif col in ['Age', 'RestingBP', 'MaxHR', 'Cholesterol']:
            # Numéricas enteras
            mu = df_base[col].mean()
            sigma = df_base[col].std()
            df_out[col] = np.random.normal(mu, sigma, n).clip(df_base[col].min(), df_base[col].max()).round().astype(int)

        elif col == 'Oldpeak':
            # Decimal controlado
            mu = df_base[col].mean()
            sigma = df_base[col].std()
            df_out[col] = np.random.normal(mu, sigma, n).clip(df_base[col].min(), df_base[col].max()).round(2)

        else:
            # Cualquier otra continua → como float, por defecto
            mu = df_base[col].mean()
            sigma = df_base[col].std()
            df_out[col] = np.random.normal(mu, sigma, n).clip(df_base[col].min(), df_base[col].max())

    return df_out

# Generar 500 sanos y 500 enfermos
sanos_fake = generar_sinteticos_mejorado(df_sano.drop(columns='HeartDisease'), n=500)
enfermos_fake = generar_sinteticos_mejorado(df_enfermo.drop(columns='HeartDisease'), n=500)

sanos_fake['HeartDisease'] = 0
enfermos_fake['HeartDisease'] = 1

# Concatenar y mezclar
df_sintetico = pd.concat([sanos_fake, enfermos_fake], ignore_index=True)
df_final = pd.concat([df, df_sintetico], ignore_index=True)
df_final = df_final.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"Número de muestras originales: {df.shape[0]}")
print(f"Número de muestras después de aumentar: {df_final.shape[0]}")

# Guardar si quieres
df_final.to_csv("heart_data_aumentado.csv", index=False)

In [ ]:
import pandas as pd
import numpy as np

# Cargar el dataset original
df = pd.read_csv("heart_data_aumentado.csv")

# --- Paso 1: Crear la variable multiclase 'HeartRiskLevel' ---
def clasificar_riesgo(row):
    score = 0
    if row['Cholesterol'] > 240:
        score += 1
    if row['RestingBP'] > 140:
        score += 1
    if row['Oldpeak'] > 2.0:
        score += 1
    if row['Age'] > 60:
        score += 1
    if row['FastingBS'] == 1:
        score += 1
    if row['ExerciseAngina_Y'] == 1:
        score += 1

    if score >= 3:
        return 2  # Alto riesgo
    elif score == 2:
        return 1  # Riesgo moderado
    else:
        return 0  # Saludable

df["HeartRiskLevel"] = df.apply(clasificar_riesgo, axis=1)

# --- Paso 2: Calcular cuántos datos hacen falta para equilibrar ---
class_counts = df["HeartRiskLevel"].value_counts().sort_index()
max_class_size = class_counts.max()
samples_needed = {cls: max_class_size - count for cls, count in class_counts.items() if count < max_class_size}

# --- Paso 3: Generar datos sintéticos por clase ---
def generar_sinteticos_equilibrado(df_base, n=500):
    df_out = pd.DataFrame()

    for col in df_base.columns:
        valores_unicos = df_base[col].nunique()

        if valores_unicos == 2 and df_base[col].dropna().isin([0, 1]).all():
            p = df_base[col].mean()
            df_out[col] = np.random.binomial(1, p, n).astype(int)

        elif col in ['Age', 'RestingBP', 'MaxHR', 'Cholesterol']:
            mu = df_base[col].mean()
            sigma = df_base[col].std()
            df_out[col] = np.random.normal(mu, sigma, n).clip(df_base[col].min(), df_base[col].max()).round().astype(int)

        elif col == 'Oldpeak':
            mu = df_base[col].mean()
            sigma = df_base[col].std()
            df_out[col] = np.random.normal(mu, sigma, n).clip(df_base[col].min(), df_base[col].max()).round(2)

        else:
            mu = df_base[col].mean()
            sigma = df_base[col].std()
            df_out[col] = np.random.normal(mu, sigma, n).clip(df_base[col].min(), df_base[col].max())

    return df_out

sinteticos = []
for clase, cantidad in samples_needed.items():
    base = df[df["HeartRiskLevel"] == clase].drop(columns=["HeartDisease", "HeartRiskLevel"])
    sint = generar_sinteticos_equilibrado(base, cantidad)
    sint["HeartRiskLevel"] = clase
    sinteticos.append(sint)

# --- Paso 4: Concatenar, mezclar y guardar ---
df_balanceado = pd.concat([df] + sinteticos, ignore_index=True).sample(frac=1, random_state=42).reset_index(drop=True)

# (Opcional) Guardar a archivo
df_balanceado.to_csv("heart_data_equilibrado_multiclase_2.csv", index=False)

# Comprobación de equilibrio
print("Distribución final de clases:")
print(df_balanceado["HeartRiskLevel"].value_counts().sort_index())